In [ ]:
import os
import pathlib
import time
import datetime
from matplotlib import pyplot as plt
from IPython import display
import tensorflow as tf

# Preparação dos dados

In [ ]:
tf.random.set_seed(789)

In [ ]:
dataset = 'maps' # cityscapes, maps, edges2shoes, edges2handbags, facades, night2day

In [ ]:
arquivo_dataset = "{}.tar.gz".format(dataset)
url_dataset = "https://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{}.tar.gz".format(dataset)
print("Baixando dataset {}...".format(dataset))
download_zip = tf.keras.utils.get_file(arquivo_dataset, origin=url_dataset, extract=True)

In [ ]:
download_zip = pathlib.Path(download_zip)
caminho = download_zip.parent/dataset

In [ ]:
caminho

In [ ]:
list(caminho.parent.iterdir())

In [ ]:
random_id = 99 # tf.random.uniform(shape=[], minval=1, maxval=1096, dtype=tf.int64).numpy()
amostra = tf.io.read_file(str(caminho/'train/{}.jpg'.format(random_id)))
amostra = tf.image.decode_jpeg(amostra)
print(amostra.shape)
plt.figure(figsize=(10,10))
plt.imshow(amostra)
plt.title("Imagem de exemplo {}".format(random_id))

# Funções para pré-processamento dos dados

In [ ]:
def carregar_imagem(img_arquivo):
  img = tf.io.read_file(img_arquivo)
  img = tf.io.decode_jpeg(img)
  img = tf.image.resize(img, [256, 512])

  largura = tf.shape(img)[1]
  largura = largura // 2

  imagem_original = img[:, :largura, :]
  imagem_transformada = img[:, largura:, :]

  imagem_original = tf.cast(imagem_original, tf.float32)
  imagem_transformada = tf.cast(imagem_transformada, tf.float32)

  return imagem_original, imagem_transformada

In [ ]:
imagem_original, imagem_transformada = carregar_imagem(str(caminho/'train/{}.jpg'.format(random_id)))

plt.figure(figsize=(10,10))
plt.subplot(121)
plt.title('Imagem Original')
plt.imshow(imagem_original/255.0)
plt.subplot(122)
plt.title('Imagem Transformada')
plt.imshow(imagem_transformada/255.0)

In [ ]:
quantidade_treino = tf.data.Dataset.list_files(str(caminho/'train/*.jpg'))
quantidade_treino

In [ ]:
quantidade_treino = len(list(quantidade_treino))
quantidade_treino

In [ ]:
buffer_size = quantidade_treino
batch_size = 1
img_largura = 256
img_altura = 256

In [ ]:
def redimencionar_imagem(imagem_original, imagem_transformada, altura, largura):
  imagem_original = tf.image.resize(imagem_original, [altura, largura], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  imagem_transformada = tf.image.resize(imagem_transformada, [altura, largura], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return imagem_original, imagem_transformada

In [ ]:
def normalizar_pixels(imagem_original, imagem_transformada):
  imagem_original = (imagem_original / 127.5) - 1
  imagem_transformada = (imagem_transformada / 127.5) - 1

  return imagem_original, imagem_transformada

In [ ]:
def crop_aleatorio(imagem_original, imagem_transformada):
  imagem_empilhada = tf.stack([imagem_original, imagem_transformada], axis=0)
  imagem_cortada = tf.image.random_crop(imagem_empilhada, size=[2, img_altura, img_largura, 3])

  return imagem_cortada[0], imagem_cortada[1]

In [ ]:
@tf.function()
def jitter_aleatorio(imagem_original, imagem_transformada):
  imagem_original, imagem_transformada = redimencionar_imagem(imagem_original, imagem_transformada, 286, 286)
  imagem_original, imagem_transformada = crop_aleatorio(imagem_original, imagem_transformada)

  if tf.random.uniform(()) > 0.5:
    imagem_original = tf.image.flip_left_right(imagem_original)
    imagem_transformada = tf.image.flip_left_right(imagem_transformada)

  return imagem_original, imagem_transformada

In [ ]:
plt.figure(figsize=(10,6))
for i in range(6):
  j_original, j_transformada = jitter_aleatorio(imagem_original, imagem_transformada)
  plt.subplot(2,3,i+1)
  plt.imshow(j_original/255.0)
  plt.axis('off')
plt.show()

# Carregamento do dataset

In [ ]:
def carrega_img_treinamento(img_arquivo):
  imagem_original, imagem_transformada = carregar_imagem(img_arquivo)
  imagem_original, imagem_transformada = jitter_aleatorio(imagem_original, imagem_transformada)
  imagem_original, imagem_transformada = normalizar_pixels(imagem_original, imagem_transformada)

  return imagem_original, imagem_transformada

In [ ]:
def carrega_img_teste(img_arquivo):
  imagem_original, imagem_transformada = carregar_imagem(img_arquivo)
  imagem_original, imagem_transformada = redimencionar_imagem(imagem_original, imagem_transformada, img_altura, img_largura)
  imagem_original, imagem_transformada = normalizar_pixels(imagem_original, imagem_transformada)

  return imagem_original, imagem_transformada

In [ ]:
dataset_treino = tf.data.Dataset.list_files(str(caminho/'train/*.jpg'))
dataset_treino = dataset_treino.map(carrega_img_treinamento, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset_treino = dataset_treino.shuffle(buffer_size)
dataset_treino = dataset_treino.batch(batch_size)
dataset_treino

In [ ]:
try:
  dataset_teste = tf.data.Dataset.list_files(str(caminho/'test/*.jpg'))
except tf.errors.InvalidArgumentError:
  dataset_teste = tf.data.Dataset.list_files(str(caminho/'val/*.jpg'))

dataset_teste = dataset_teste.map(carrega_img_teste)
dataset_teste = dataset_teste.batch(batch_size)
dataset_teste


# Gerador

In [ ]:
def encode(filters, size, apply_instancenorm=True):
  initializer = tf.random_normal_initializer(0.,0.02)

  camada = tf.keras.Sequential()
  camada.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))

  if apply_instancenorm:
    camada.add(tf.keras.layers.BatchNormalization())

  camada.add(tf.keras.layers.LeakyReLU())

  return camada

In [ ]:
down_model = encode(64, 4, apply_instancenorm=False)
down_resultado = down_model(tf.expand_dims(imagem_original, 0))
print(down_resultado.shape)

In [ ]:
def decode(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0.,0.02)

  camada = tf.keras.Sequential()
  camada.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))

  camada.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
    camada.add(tf.keras.layers.Dropout(0.5))

  camada.add(tf.keras.layers.ReLU())

  return camada

In [ ]:
up_model = decode(64, 4)
up_resultado = up_model(down_resultado)
print(up_resultado.shape)

In [ ]:
def gerador_nn():
    inputs = tf.keras.layers.Input(shape=[256,256,3])

    down_stack = [
        encode(64, 4, apply_instancenorm=False), # (bs, 128, 128, 64)
        encode(128, 4), # (bs, 64, 64, 128)
        encode(256, 4), # (bs, 32, 32, 256)
        encode(512, 4), # (bs, 16, 16, 512)
        encode(512, 4), # (bs, 8, 8, 512)
        encode(512, 4), # (bs, 4, 4, 512)
        encode(512, 4), # (bs, 2, 2, 512)
        encode(512, 4), # (bs, 1, 1, 512)
    ]

    up_stack = [
        decode(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
        decode(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
        decode(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
        decode(512, 4), # (bs, 16, 16, 1024)
        decode(256, 4), # (bs, 32, 32, 512)
        decode(128, 4), # (bs, 64, 64, 256)
        decode(64, 4), # (bs, 128, 128, 128)
    ]

    canais_saida = 3
    initializer = tf.random_normal_initializer(0.,0.02)
    ultima_camada = tf.keras.layers.Conv2DTranspose(canais_saida, 4, strides=2, padding='same', kernel_initializer=initializer, activation='tanh') # (bs, 256, 256, 3)

    x = inputs

    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = ultima_camada(x)

    return tf.keras.Model(inputs=inputs, outputs=x)
    

In [ ]:
gerador = gerador_nn()
tf.keras.utils.plot_model(gerador, show_shapes=True, dpi=64)

In [ ]:
g_saida = gerador(imagem_original[tf.newaxis,...], training=False)
plt.imshow(g_saida[0,...])

## Error

In [ ]:
lr = 2e-4
beta_1, beta_2 = 0.5, 0.999
lambda_ciclo = 10

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def loss_gerador(discriminador_fake_output, gerador_output, target):
  gan_loss = loss(tf.ones_like(discriminador_fake_output), discriminador_fake_output)

  l1_loss = tf.reduce_mean(tf.abs(target - gerador_output))

  total_loss = gan_loss + (lambda_ciclo * l1_loss)

  return total_loss, gan_loss, l1_loss

# Discriminador

In [ ]:
def discriminador():
  initializer = tf.random_normal_initializer(0.,0.02)

  original = tf.keras.layers.Input(shape=[256,256,3], name='img_original')
  transformada = tf.keras.layers.Input(shape=[256,256,3], name='img_transformada')

  entrada = tf.keras.layers.concatenate([original, transformada]) # (bs, 256, 256, channels*2)

  down1 = encode(64, 4, False)(entrada) # (bs, 128, 128, 64)
  down2 = encode(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = encode(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  ultima_camada = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[original, transformada], outputs=ultima_camada)

In [ ]:
discriminador = discriminador()
tf.keras.utils.plot_model(discriminador, show_shapes=True, dpi=64)

In [ ]:
d_saida = discriminador([imagem_original[tf.newaxis,...], g_saida], training=False)
plt.imshow(d_saida[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')

## Perda do discriminador

In [ ]:
def loss_discriminador(discriminador_real_output, discriminador_fake_output):
  real_loss = loss(tf.ones_like(discriminador_real_output), discriminador_real_output)

  fake_loss = loss(tf.zeros_like(discriminador_fake_output), discriminador_fake_output)

  total_loss = real_loss + fake_loss

  return total_loss

# Otimizadores

In [ ]:
optimizador_gerador = tf.keras.optimizers.Adam(lr, beta_1, beta_2)
optimizador_discriminador = tf.keras.optimizers.Adam(lr, beta_1, beta_2)

# checkpoints (Object-based saving)

In [ ]:
checkpoint_dir = 'pix2pix/treinamento_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=optimizador_gerador,
                                  discriminador_optimizer=optimizador_discriminador,
                                  generator=gerador,
                                  discriminador=discriminador)

# Geração de imagens

In [ ]:
def gerar_imagens(modelo, teste_entrada, real, etapa = None):
  pred = modelo(teste_entrada, training=True)
  plt.figure(figsize=(12,8))

  display_list = [teste_entrada[0], real[0], pred[0]]
  titulo = ['Imagem de entrada', 'Real (ground truth)', 'Imagem gerada (Fake)']

  for i in range(3):
    plt.subplot(1,3,i+1)
    plt.title(titulo[i])
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  if etapa != None:
    plt.savefig('pix2pix/img_{}.png'.format(etapa), bbox_inches='tight')
  plt.show()

In [ ]:
for exemplo_input, exemplo_target in dataset_teste.take(7):
  gerar_imagens(gerador, exemplo_input, exemplo_target)

# Treinamento

In [ ]:
caminho_log = 'pix2pix/logs/'
metricas = tf.summary.create_file_writer(caminho_log + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def etapa_treinamento(img_entrada, real, etapa):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gerador_output = gerador(img_entrada, training=True)

    discriminador_real_output = discriminador([img_entrada, real], training=True)
    discriminador_fake_output = discriminador([img_entrada, gerador_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = loss_gerador(discriminador_fake_output, gerador_output, real)
    disc_loss = loss_discriminador(discriminador_real_output, discriminador_fake_output)

  gerador_gradientes = gen_tape.gradient(gen_total_loss, gerador.trainable_variables)
  discriminador_gradientes = disc_tape.gradient(disc_loss, discriminador.trainable_variables)

  optimizador_gerador.apply_gradients(zip(gerador_gradientes, gerador.trainable_variables))
  optimizador_discriminador.apply_gradients(zip(discriminador_gradientes, discriminador.trainable_variables))

  with metricas.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=etapa//1000)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=etapa//1000)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=etapa//1000)
    tf.summary.scalar('disc_loss', disc_loss, step=etapa//1000)

In [ ]:
def treinar(base_treinamento, base_teste, etapas):
  exemplo_input, exemplo_target = next(iter(base_teste.take(1)))
  inicio = time.time()

  for etapa, (img_entrada, real) in base_treinamento.repeat().take(etapas).enumerate():
    if etapa % 1000 == 0:
      display.clear_output(wait=True)

      if (etapa != 0):
        tempo = time.time() - inicio
        print ('Tempo decorrido: {} segundo'.format(tempo))

      inicio = time.time()

      gerar_imagens(gerador, exemplo_input, exemplo_target, etapa)
      print ('Etapa: {}'.format(etapa))

    etapa_treinamento(img_entrada, real, etapa)
    if (etapa + 1) % 10 == 0:
      print ('.', end='', flush=True)
    if (etapa + 1) % 5000 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
      gerador.set_weights(gerador.get_weights())

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {caminho_log}

In [ ]:
treinar(dataset_treino, dataset_teste, etapas=5000)

In [ ]:
gerador.save_weights('pix2pix/gerador.h5')

# Restaurando o último checkpoint para testes

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
modelo_pre_treinado = gerador_nn()
modelo_pre_treinado.load_weights('pix2pix/gerador.h5')

In [ ]:
for satelite, mapa in dataset_teste.take(5):
  gerar_imagens(modelo_pre_treinado, satelite, mapa)